In [35]:
import pandas as pd


def preprocess_data(data_path: str) -> pd.DataFrame:
    df = pd.read_csv(data_path)
    return df


In [36]:
data = preprocess_data("../data/Airbnb_Open_Data.csv")

/var/folders/4f/jhvwxkwj6cl927nhqr00hnqw0000gn/T/ipykernel_97317/2387075989.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [37]:
!pwd

/Users/adrienbufort/Documents/workspace/LinearMerger/scripts


In [38]:
# we create a first dataset with only the float columns
float_columns = data.select_dtypes(include=['float64']).columns
float_data = data[float_columns]




In [39]:
float_data

,lat,long,Construction year,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
0,40.64749,-73.97237,2020.0,10.0,9.0,0.21,4.0,6.0,286.0
1,40.75362,-73.98377,2007.0,30.0,45.0,0.38,4.0,2.0,228.0
2,40.80902,-73.94190,2005.0,3.0,0.0,NaN,5.0,1.0,352.0
3,40.68514,-73.95976,2005.0,30.0,270.0,4.64,4.0,1.0,322.0
4,40.79851,-73.94399,2009.0,10.0,9.0,0.10,3.0,1.0,289.0
...,...,...,...,...,...,...,...,...,...
102594,40.70862,-73.94651,2003.0,1.0,0.0,NaN,3.0,1.0,227.0
102595,40.80460,-73.96545,2016.0,1.0,1.0,0.02,2.0,2.0,395.0
102596,40.67505,-73.98045,2009.0,3.0,0.0,NaN,5.0,1.0,342.0
102597,40.74989,-73.93777,2015.0,2.0,5.0,0.10,3.0,1.0,386.0


In [40]:
data.shape

(102599, 26)

In [41]:

# now we keep only the columns :
# - price
# - neighbourhood group
# - instant_bookable
# - cancellation_policy
# - room type
# - service fee
interesting_columns = ['price', 'neighbourhood group', 'instant_bookable', 'cancellation_policy', 'room type', 'service fee']
object_data = data[interesting_columns]



# we convert the price and service fee to float
# we remove the $ from the price and service fee
# replace , by .    
object_data['price'] = object_data['price'].str.replace('$', '').str.replace(',', '.').astype(float)
object_data['service fee'] = object_data['service fee'].str.replace('$', '').str.replace(',', '.').astype(float)

object_data

# then for every object column, we convert the values to categorical
for column in object_data.columns:
    if object_data[column].dtype == 'object':
        object_data[column] = object_data[column].astype('category')

# then we replace categorical type values by their code for every object column
for column in object_data.columns:
    if object_data[column].dtype == 'category':
        object_data[column] = object_data[column].cat.codes

object_data


/var/folders/4f/jhvwxkwj6cl927nhqr00hnqw0000gn/T/ipykernel_97317/1733692789.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_data['price'] = object_data['price'].str.replace('$', '').str.replace(',', '.').astype(float)
/var/folders/4f/jhvwxkwj6cl927nhqr00hnqw0000gn/T/ipykernel_97317/1733692789.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_data['service fee'] = object_data['service fee'].str.replace('$', '').str.replace(',', '.').astype(float)
/var/folders/4f/jhvwxkwj6cl927nhqr00hnqw

,price,neighbourhood group,instant_bookable,cancellation_policy,room type,service fee
0,966.000,1,0,2,2,193.0
1,142.000,2,0,1,0,28.0
2,620.000,2,1,0,2,124.0
3,368.000,1,1,1,0,74.0
4,204.000,2,0,1,0,41.0
...,...,...,...,...,...,...
102594,844.000,1,0,0,2,169.0
102595,837.000,2,1,1,2,167.0
102596,988.000,1,1,1,2,198.0
102597,546.000,3,1,2,0,109.0


In [42]:
# now we merge the two datasets
merged_data = pd.concat([float_data, object_data], axis=1)

merged_data


,lat,long,Construction year,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365,price,neighbourhood group,instant_bookable,cancellation_policy,room type,service fee
0,40.64749,-73.97237,2020.0,10.0,9.0,0.21,4.0,6.0,286.0,966.000,1,0,2,2,193.0
1,40.75362,-73.98377,2007.0,30.0,45.0,0.38,4.0,2.0,228.0,142.000,2,0,1,0,28.0
2,40.80902,-73.94190,2005.0,3.0,0.0,NaN,5.0,1.0,352.0,620.000,2,1,0,2,124.0
3,40.68514,-73.95976,2005.0,30.0,270.0,4.64,4.0,1.0,322.0,368.000,1,1,1,0,74.0
4,40.79851,-73.94399,2009.0,10.0,9.0,0.10,3.0,1.0,289.0,204.000,2,0,1,0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,40.70862,-73.94651,2003.0,1.0,0.0,NaN,3.0,1.0,227.0,844.000,1,0,0,2,169.0
102595,40.80460,-73.96545,2016.0,1.0,1.0,0.02,2.0,2.0,395.0,837.000,2,1,1,2,167.0
102596,40.67505,-73.98045,2009.0,3.0,0.0,NaN,5.0,1.0,342.0,988.000,1,1,1,2,198.0
102597,40.74989,-73.93777,2015.0,2.0,5.0,0.10,3.0,1.0,386.0,546.000,3,1,2,0,109.0


In [44]:
merged_data = merged_data.dropna()


In [45]:
# save the merged data to a parquet file
merged_data.to_parquet("../data/merged_data.parquet")